<a href="https://colab.research.google.com/github/Ecemm1/python-mini--projects/blob/main/Otomasyon_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import openpyxl
import pandas as pd
import os

# Use forward slashes for the file path
csv_dosya_yolu = "satislar.csv"
xlsx_dosya_yolu = "satislar.xlsx"

# Read the CSV file with semicolon delimiter
try:
    df = pd.read_csv(csv_dosya_yolu, delimiter=';')
    print("CSV file read successfully.")
    print(df.head())

    # Clean the 'BİRİM FİYAT(TL)' column
    # Remove '₺', '.', ' ', and '00;', and replace ',' with '.' before converting to numeric
    df['BİRİM FİYAT(TL)'] = df['BİRİM FİYAT(TL)'].astype(str).str.replace('₺', '', regex=False).str.replace('.', '', regex=False).str.replace(' 00;', '', regex=False).str.replace(',', '.', regex=False)
    df['BİRİM FİYAT(TL)'] = pd.to_numeric(df['BİRİM FİYAT(TL)'], errors='coerce')

    # Clean the 'ADET' column
    df['ADET'] = pd.to_numeric(df['ADET'], errors='coerce')


    # Convert to Excel
    df.to_excel(xlsx_dosya_yolu, index=False)
    print(f"{xlsx_dosya_yolu} file created successfully.")

except FileNotFoundError:
    print(f"{csv_dosya_yolu} not found.")
except Exception as e:
    print(f"An error occurred while processing the CSV file: {e}")

# Process the Excel file
try:
    workbook = openpyxl.load_workbook(xlsx_dosya_yolu)
    print("Excel file loaded successfully.")

    # Assuming the data is in the first sheet
    sheet = workbook.active
    print(f"'{sheet.title}' sheet selected.")

    # Access cell values
    a1_degeri = sheet["A1"].value
    print(f"A1 cell value: {a1_degeri}")
    # Access B2 value directly from DataFrame for clarity after cleaning
    if 'ADET' in df.columns and len(df) > 1:
        b2_degeri = df.loc[1, 'ADET']
        print(f"B2 cell value (from DataFrame after cleaning): {b2_degeri}")
    else:
        print("Could not access B2 value from DataFrame.")


    # Create or select the "Rapor" sheet
    if "Rapor" in workbook.sheetnames:
        rapor_sayfasi = workbook["Rapor"]
        print("'Rapor' sheet selected.")
    else:
        rapor_sayfasi = workbook.create_sheet("Rapor")
        print("New 'Rapor' sheet created.")

    # Add headers to the report sheet
    rapor_basliklari = ["ÜRÜN ADI", "ADET", "BİRİM FİYAT(TL)", "TOPLAM TUTAR(TL)"]
    rapor_sayfasi.append(rapor_basliklari)
    print("Reading raw data and performing calculations...")

    # Process data from the original sheet and append to the report sheet
    for row in sheet.iter_rows(min_row=2, values_only=True):
        # Ensure there are enough values in the row before unpacking
        if len(row) >= 3:
            urun_adi, adet, birim_fiyat = row[:3]
            # Ensure adet and birim_fiyat are numeric before multiplication
            try:
                # Use the cleaned numeric values directly if possible, otherwise convert
                adet = float(adet) if adet is not None else 0
                birim_fiyat = float(birim_fiyat) if birim_fiyat is not None else 0
                toplam_tutar = adet * birim_fiyat
                yeni_satir_verisi = [urun_adi, adet, birim_fiyat, toplam_tutar]
                rapor_sayfasi.append(yeni_satir_verisi)
            except (ValueError, TypeError):
                print(f"Skipping row due to non-numeric data: {row}")
        else:
            print(f"Skipping row with insufficient data: {row}")


    # Save the workbook
    kaydedilecek_isim = "satislar_raporlu.xlsx"
    workbook.save(kaydedilecek_isim)
    print(f"Saved to {kaydedilecek_isim}.")

except FileNotFoundError:
    print(f"Excel file {xlsx_dosya_yolu} not found.")
except Exception as e:
    print(f"An error occurred while processing the Excel file: {e}")

CSV file read successfully.
  ÜRÜN ADI  ADET BİRİM FİYAT(TL)  Unnamed: 3
0   Laptop   5.0      ₺15.000,00         NaN
1   Klavye  20.0         ₺450,00         NaN
2  Monitör  10.0       ₺3.200,00         NaN
3    Mouse  50.0         ₺200,00         NaN
4      NaN   NaN             NaN         NaN
satislar.xlsx file created successfully.
Excel file loaded successfully.
'Sheet1' sheet selected.
A1 cell value: ÜRÜN ADI
B2 cell value (from DataFrame after cleaning): 20.0
New 'Rapor' sheet created.
Reading raw data and performing calculations...
Saved to satislar_raporlu.xlsx.


In [11]:
try:
    # Load the workbook
    workbook_rapor = openpyxl.load_workbook("satislar_raporlu.xlsx")
    print("satislar_raporlu.xlsx loaded successfully.")

    # Select the "Rapor" sheet
    if "Rapor" in workbook_rapor.sheetnames:
        rapor_sayfasi = workbook_rapor["Rapor"]
        print("'Rapor' sheet selected.")

        # Read the sheet data into a pandas DataFrame
        # Use the first row as headers
        data = rapor_sayfasi.values
        columns = next(data)
        df_rapor = pd.DataFrame(data, columns=columns)

        # Display the DataFrame
        print("\nContent of 'Rapor' sheet:")
        display(df_rapor)

    else:
        print("'Rapor' sheet not found in the workbook.")

except FileNotFoundError:
    print("satislar_raporlu.xlsx not found. Please ensure the previous steps were completed successfully.")
except Exception as e:
    print(f"An error occurred while reading the 'Rapor' sheet: {e}")

satislar_raporlu.xlsx loaded successfully.
'Rapor' sheet selected.

Content of 'Rapor' sheet:


,ÜRÜN ADI,ADET,BİRİM FİYAT(TL),TOPLAM TUTAR(TL)
0,Laptop,5,15000,75000
1,Klavye,20,450,9000
2,Monitör,10,3200,32000
3,Mouse,50,200,10000
4,None,0,0,0
5,None,0,0,0
6,None,0,0,0
